# EDA

In [1]:
# Import libraries

import pandas as pd
import numpy as np

In [ ]:
def get_hashtag_counts(json):
    
    # Create empty dict where values are stored
    hashtag_count = defaultdict(int)
    
    # Iterate over all tweets, retrieve their hashtags and increment the counter by one
    for tweet in json["data"]:
        try:
            hashtags = tweet["entities"]["hashtags"]
            for hashtag in hashtags:
                hashtag_count[hashtag["tag"]] +=1
        except KeyError:
            continue
            
    return hashtag_count

def get_annotation_counts(json):
    
    annotation_count = defaultdict(int)

    for tweet in json["data"]:
        try:
            annotations = tweet["entities"]["annotations"]
            for annotation in annotations:
                annotation_count[annotation["normalized_text"]] +=1
        except KeyError:
            continue
            
    return annotation_count

In [3]:
df_tweets = pd.read_csv(r"C:\Users\jawo19ad\Dropbox (CBS)\Master thesis data\df_tweets.csv", index_col=0)
df_tweets.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,source,text,lang,id,created_at,author_id,entities.mentions,entities.hashtags,entities.urls,...,public_metrics.quote_count,entities.annotations,in_reply_to_user_id,referenced_tweets,geo.place_id,geo.coordinates.type,geo.coordinates.coordinates,withheld.copyright,withheld.country_codes,entities.cashtags
0,0,Twitter for Android,There are too many reliable reports and first-...,en,1350390669043499013,2021-01-16T10:33:19.000Z,1327278886380515328,"[{'start': 178, 'end': 187, 'username': 'JoeBi...","[{'start': 164, 'end': 172, 'tag': 'Eritrea'},...","[{'start': 250, 'end': 273, 'url': 'https://t....",...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Twitter Web App,"Despite repeated requests, @Refugees and other...",en,1350390657576300544,2021-01-16T10:33:16.000Z,1323903491044188161,"[{'start': 27, 'end': 36, 'username': 'Refugee...","[{'start': 166, 'end': 173, 'tag': 'Tigray'}, ...","[{'start': 252, 'end': 275, 'url': 'https://t....",...,0,"[{'start': 101, 'end': 108, 'probability': 0.3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Twitter for Android,"“When the air bombing and the attacks began, I...",en,1350390643986599937,2021-01-16T10:33:13.000Z,1324130252008816640,"[{'start': 190, 'end': 199, 'username': 'JoeBi...","[{'start': 231, 'end': 247, 'tag': 'BidenTakeA...","[{'start': 248, 'end': 271, 'url': 'https://t....",...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Twitter Web App,"""ongoing insecurity &amp; allegations of grave...",en,1350390618695020546,2021-01-16T10:33:07.000Z,1112761003,"[{'start': 144, 'end': 147, 'username': 'UN'},...","[{'start': 200, 'end': 215, 'tag': 'TigrayGeno...","[{'start': 246, 'end': 269, 'url': 'https://t....",...,0,"[{'start': 127, 'end': 133, 'probability': 0.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Twitter for iPhone,"MOTHER \n\nMother, I do not cry who cries, \nm...",en,1350390607928295424,2021-01-16T10:33:04.000Z,1212824799107375105,NaN,NaN,NaN,...,0,NaN,1.212825e+18,"[{'type': 'replied_to', 'id': '135038931103271...",NaN,NaN,NaN,NaN,NaN,NaN
